<a href="https://colab.research.google.com/github/eusebiogerman/WEBSCRAPING/blob/master/nba_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import unittest
import time
import sys
import os
from os import path
import traceback
import requests
import logging

from console_progressbar import ProgressBar
from bs4 import BeautifulSoup , Comment 
from lxml import etree

#test
def newtag(bs : BeautifulSoup, tagn : str ,text: str) :
    tag = bs.new_tag(tagn)
    tag.string = text 
    return tag

def get_trace_value(plist):
    trace = ''
    for p in plist:
        trace = trace + p + '/n'
    return trace

def load_nba_schedule(year):
    
    #BASE_URL = "https://www.baseball-reference.com" 
    BASE_URL = "https://www.basketball-reference.com/"
    link = f"{BASE_URL}/leagues/NBA_{year}_games.html"
    link2 = ''
    archivo = ''
    fecha = ''
    schedule = []
    gdir = str(year)
    dummycnt = 1
    
    try:
        #crear el repots
        if not (path.isdir('sessons')):
            os.mkdir('sessons')

        #carga el archivo de los datos
        #file = open("mlb_game.txt","a")

        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
        
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            print(f"Schedule {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')
            div_schedule = soup.find_all('div', {'id','div_schedule'})
            table = soup.find_all('table')
            tb = table[0].find_all('tbody')
            trs = tb[0].find_all('tr')

            for tr in trs:
                tds = tr.find_all('td', {'class','center'})
                for td in tds:
                    a = td.find_all('a')
                    if (a) :
                       dict = {}
                       fecha = a[0].get('href').split('/')[2][0:9]
                       archivo = f"sessons/{gdir}.txt"
                       link2 =  f"{BASE_URL}{a[0].get('href')}"
                       dict ={'link' : link2, 'archivo' : archivo, 'fecha' : fecha }  
                       schedule.append(dict)
        return schedule
    except ValueError:
            print("Error de Valor")
            traceback.print_exception(*sys.exc_info())
            return schedule
    except:    
            print("Error de general") 
            traceback.print_exception(*sys.exc_info())   
            return schedule                     


def load_nba_games(link, archivo, gamedate):
    
    logfile = archivo.split('/')[1].replace('txt','log')
    logfile = archivo.split('/')[0] +'/'+ logfile
    logging.basicConfig(filename = 'test.log', enconding ='utf-8',level=logging.INFO)

    try:

        #carga el archivo de los datos
        #file = open(archivo, 'a')

        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
        
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            logging.info(f"Process {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')

            #tag escondido para listar los equipos
            comments = soup.find_all(text=lambda text:isinstance(text, Comment))
            htm = str([htm for htm in comments if "div_other_scores" in htm]);

            htmwrap = BeautifulSoup(htm,'html.parser')
            te_as =  htmwrap.find_all('div',class_ ='game_summary nohover current')[0].find_all('a')
            teams = [teams.text for teams in te_as]
            del teams[1]
 
            #codigo de los equipos
            div_qs = soup.find_all('div',class_='filter switcher')
            divbox = div_qs[0].find_all('div')
            divbox = [x.a.get('data-show').split(',') for x in divbox]
            del divbox[0]
            #print(divbox)  
            for box_out in divbox:
                print(box_out)
                for box in box_out:
                    print(f"div_{box[1:]}-basic")
                    divb = f"{box[1:]}-basic"
                    #print(divb)
                    dbx = soup.find_all('div',{'id':f"div_{divb}"})
                    tb   = dbx[1].find_all('table',{'id':divb})
                    trs  = tb[0].find_all('tr')
                    for tr in trs :
                        tds  = tr.find_all('td')
                        if (tds) :
                            tds  = [x.text for x in tds]
                            print(tds)

   
        
            '''
            teams = uls[0].find_all('a')
            home  = teams[1]['href'].split('/')[2]
            visi  = teams[2]['href'].split('/')[2]

            #saca el tag escondido
            comments = soup.find_all(text=lambda text:isinstance(text, Comment))
            htm = str([htm for htm in comments if "div_play_by_play" in htm]);
            if(htm):                          
                div = BeautifulSoup(htm,'html.parser')
                tbody  = div.find_all("tbody")
                rows = tbody[0].find_all('tr', class_ =['top_inning','bottom_inning'])
               
                for row in rows:
                    if row != ['']:
                       thtext = row.find_all('th')[0].text                     
                       row.insert(0, newtag(div,'td',thtext))
                       row.insert(0, newtag(div,'td',visi))
                       row.insert(0, newtag(div,'td',home))
                       row.insert(0, newtag(div,'td',gamedate))                     

                      # print(row)
                       cols=row.find_all('td')
                       cols=[" ".join(x.text.replace(',','|').upper().split()) for x in cols]
                       file.write(" , ".join(cols)+'\n')
                       #print(cols)  
            file.close  
            '''
    except ValueError:
            #file.close 
            print("Error de Valor : load_mlb_games ")
            logging.error(get_trace_value(sys.exc_info()))
    except:    
            #file.close                     
            print("Error de general : load_mlb_games") 
            logging.error(get_trace_value(sys.exc_info()))




'''
sche = load_nba_schedule(2018) 
for es in sche:
    print(es)
'''
score  = {'link': 'https://www.basketball-reference.com//boxscores/201710230DEN.html', 'archivo': 'sessons/2018.txt', 'fecha': '201710230'}
load_nba_games(score['link'],score['archivo'],score['fecha'])


# https://www.basketball-reference.com/leagues/NBA_2018_games.html
# https://www.basketball-reference.com/teams/DEN/2021_games.html
# https://www.basketball-reference.com/boxscores/202012230POR.html

    

['.box-WAS-q1', '.box-DEN-q1']
div_box-WAS-q1-basic
['8:31', '1', '2', '.500', '0', '1', '.000', '0', '0', '', '0', '1', '1', '0', '0', '0', '0', '1', '2', '-3']
['8:54', '1', '3', '.333', '1', '3', '.333', '0', '0', '', '0', '1', '1', '2', '0', '0', '0', '0', '3', '-5']
['9:40', '0', '4', '.000', '0', '1', '.000', '2', '2', '1.000', '0', '0', '0', '4', '0', '0', '0', '0', '2', '-3']
['10:11', '2', '5', '.400', '2', '3', '.667', '0', '0', '', '0', '1', '1', '1', '0', '0', '0', '0', '6', '-3']
['8:54', '3', '3', '1.000', '0', '0', '', '0', '0', '', '2', '1', '3', '0', '0', '0', '1', '0', '6', '-5']
['3:06', '1', '1', '1.000', '0', '0', '', '0', '0', '', '0', '1', '1', '0', '0', '0', '0', '1', '2', '+5']
['3:06', '0', '1', '.000', '0', '1', '.000', '0', '0', '', '0', '1', '1', '1', '0', '0', '1', '0', '0', '+5']
['3:29', '0', '2', '.000', '0', '1', '.000', '0', '0', '', '0', '0', '0', '0', '0', '0', '0', '0', '0', '+3']
['2:20', '1', '1', '1.000', '1', '1', '1.000', '0', '0', '', '1', '0